# Exotic Derivatives Trading Desk Analysis

**As of 16 May 2025**, we will:
1. Fetch market data  
2. Instantiate each of our four option classes  
3. Price & (where available) compute delta  
4. Summarize hedges for the combined portfolio  


In [1]:
# Install needed packages (run this cell once)
!pip install --quiet yfinance scipy


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
# Cell 2: imports and discount-factor setup
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

from derivatives import (
    EuropeanCall,
    AmericanPut,
    UpAndInBarrierCall,
    BasketCall
)

# Flat 4% risk-free curve → discount factor function
r = 0.04
discount = lambda T: np.exp(-r * T)


In [3]:
# Cell 3: Market inputs as of 16 May 2025

trade_date = "2025-05-16"
next_day   = (
    datetime.datetime.strptime(trade_date, "%Y-%m-%d")
    + datetime.timedelta(days=1)
).strftime("%Y-%m-%d")

tickers = ["BHP.AX","CBA.AX","WES.AX","CSL.AX","WDS.AX","MQG.AX"]
df = yf.download(tickers, start=trade_date, end=next_day)["Close"]
S0 = df.loc[trade_date].to_dict()

# Placeholder implied vols
vol = {
    "BHP.AX": 0.25,
    "CBA.AX": 0.22,
    "WES.AX": 0.30,
    "CSL.AX": 0.20,
    "WDS.AX": 0.28,
    "MQG.AX": 0.23
}

print("Spot prices:", S0)
print("Volatilities:", vol)
print("Flat r:", r)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  6 of 6 completed

Spot prices: {'BHP.AX': 39.720001220703125, 'CBA.AX': 169.66000366210938, 'CSL.AX': 241.82000732421875, 'MQG.AX': 207.35000610351562, 'WDS.AX': 21.920000076293945, 'WES.AX': 82.55999755859375}
Volatilities: {'BHP.AX': 0.25, 'CBA.AX': 0.22, 'WES.AX': 0.3, 'CSL.AX': 0.2, 'WDS.AX': 0.28, 'MQG.AX': 0.23}
Flat r: 0.04


In [4]:
# Cell 4: Instantiate & price

# 1) BHP European Call
T1         = (datetime.datetime(2027,9,15) - datetime.datetime(2025,5,16)).days/365
bhp_call   = EuropeanCall(
    S0       = S0["BHP.AX"],
    K        = 0.98 * S0["BHP.AX"],
    T        = T1,
    discount = discount,
    sigma    = vol["BHP.AX"]
)
price_bhp, delta_bhp = bhp_call.price(), bhp_call.delta()

# 2) CBA American Put
T2        = (datetime.datetime(2026,5,15) - datetime.datetime(2025,5,16)).days/365
cba_put   = AmericanPut(
    S0       = S0["CBA.AX"],
    K        = 170.0,
    T        = T2,
    discount = discount,
    sigma    = vol["CBA.AX"]
)
price_cba, delta_cba = cba_put.price(), cba_put.delta()

# 3) WES Up-and-In Barrier Call
wes_bar   = UpAndInBarrierCall(
    S0       = S0["WES.AX"],
    K        = 80.0,
    T        = T1,
    discount = discount,
    sigma    = vol["WES.AX"],
    barrier  = 100.0
)
price_wes, delta_wes = wes_bar.price(), wes_bar.delta()

# 4) Four-Stock Basket Call
T3         = (datetime.datetime(2025,7,17) - datetime.datetime(2025,5,16)).days/365
basket     = BasketCall(
    S0_list    = [S0[t] for t in ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]],
    weights    = [0.10,0.35,0.15,0.40],
    K          = 175.0,
    T          = T3,
    discount   = discount,
    sigma_list = [vol[t] for t in ["BHP.AX","CSL.AX","WDS.AX","MQG.AX"]]
)
price_bask, delta_bask = basket.price(), basket.delta()

# Quick print
print(f"BHP Call     → Price: {price_bhp:.2f}, Δ: {delta_bhp:.4f}")
print(f"CBA Put      → Price: {price_cba:.2f}, Δ: {delta_cba:.4f}")
print(f"WES Barrier  → Price: {price_wes:.2f}, Δ: {delta_wes:.4f}")
print(f"Basket Call  → Price: {price_bask:.2f}, Δ: {delta_bask:.4f}")


ValueError: operands could not be broadcast together with shapes (4,) (4,252) 